In [ ]:
import os
from Bio import SeqIO
import pandas as pd
import numpy as np
from statsmodels.sandbox.stats.multicomp import multipletests
import sys
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
os.environ['R_HOME'] = "/home/cfriedline/R3/lib64/R/"
import rpy2.ipython
import rpy2.robjects as robjects
%load_ext rpy2.ipython
r = robjects.r
ri2py = robjects.conversion.ri2py

In [ ]:
%%R
library(topGO)
library(qvalue)

In [ ]:
cd ~/g/projects/black_spruce_new/

In [ ]:
%%R
load("topgo_input.Rdata")

In [ ]:
%%R
library(topGO)
gene_names = rownames(full_with_go)
cambium_interesting = rownames(dge_C)
needle_interesting = rownames(dge_N)
gene_id_2go  = readMappings(file="go_mappings.txt")
interesting = list()
interesting$cambium = cambium_interesting
interesting$needle = needle_interesting
godata = list()
gentables = list()
gentables_bh = list()
gentables_qval = list()
onts = c("BP","CC", "MF")
sigs = list()
descriptions = list()
for (i in 1:length(onts)) {
    for (j in 1:length(interesting)) {
        interest = interesting[[j]]
        gene_list <- factor(as.integer(gene_names %in% interest))
        names(gene_list) <- gene_names
        description=paste(names(interesting)[j], onts[i], sep="-")
        descriptions = append(descriptions, description)
        GOdata = new("topGOdata",
                     description=description,
                     ontology = onts[i], 
                     allGenes = gene_list, 
                     annot = annFUN.gene2GO, 
                     gene2GO = gene_id_2go,
                     nodeSize=2)
        print(GOdata)
        godata = append(godata, GOdata)
        classicFisher = runTest(GOdata, algorithm = "classic", statistic = "fisher")
        weight01Fisher = runTest(GOdata, algorithm = "weight01", statistic = "fisher")
        sigs = append(sigs, classicFisher)
        printGraph(GOdata, 
                   classicFisher, 
                   firstSigNodes = 2, 
                   fn.prefix = paste("tGO", "for", description(GOdata)), 
                   #fn.prefix = paste("tGOslim", "for", description(GOdata)), 
                   useInfo = "all")
        
        gt = GenTable(GOdata, 
                      classicFisher=classicFisher, 
                      weight01Fisher=weight01Fisher, 
                      topNodes=length(classicFisher@score), 
                      orderBy="classicFisher", numChar=1000)
        gentables = append(gentables, list(gt))
        
        fisher_p = as.numeric(gt[,"classicFisher"])
        
        gt.bh = gt[which(p.adjust(fisher_p,method="BH")<=0.05),]
        
        #print(qvalue(fisher_p))
    
        
        gt.qval = gt[which(qvalue(fisher_p)$qvalues<=0.05),]
        gentables_bh = append(gentables_bh, list(gt.bh))
        gentables_qval = append(gentables_qval, list(gt.qval))
        write.table(gt, file=paste(description(GOdata), ".txt", sep=""), row.names=F)
        write.table(gt.bh, file=paste(description(GOdata), "_bh.txt", sep=""), row.names=F)
        write.table(gt.qval, file=paste(description(GOdata), "_qval.txt", sep=""), row.names=F)  
    }
}
save.image("topgo.Rdata")